# 1. Tổng quan

```Chain-of-Thought (CoT) prompting``` là kỹ thuật hướng dẫn mô hình ngôn ngữ lớn (LLM) trình bày các bước suy luận trung gian trước khi đưa ra câu trả lời cuối cùng. Phương pháp này mô phỏng quá trình tư duy từng bước của con người, giúp cải thiện độ chính xác và khả năng giải quyết vấn đề phức tạp.

# 2. Cách thức hoạt động
* Phân rã vấn đề: Chia bài toán phức tạp thành các bước đơn giản.
* Hướng dẫn suy luận: Yêu cầu mô hình "hiển thị" quá trình suy nghĩ.
* Tổng hợp kết quả: Dựa trên các bước trung gian để đưa ra đáp án cuối.

# 3. Các phương pháp triển khai

In [1]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv

load_dotenv()

True

## 3.1. Zero-shot CoT
* Cách dùng: Thêm cụm từ kích hoạt (ví dụ: "Hãy suy nghĩ từng bước") vào prompt.
* Ưu điểm: Không cần ví dụ mẫu.

In [2]:
model = init_chat_model(
    model='gpt-4o-mini', 
    model_provider='openai',
    temperature=0.1
)


messages = [
    SystemMessage('Trong vai trò chuyên gia toán học, hãy giúp tôi giải các bài toán sau'),
    HumanMessage('Giải phương trình x^2 - 5x + 6 = 0. Hãy suy nghĩ từng bước.'),
]

response = model.invoke(messages)
print(response.content) 

Để giải phương trình bậc hai \( x^2 - 5x + 6 = 0 \), chúng ta sẽ sử dụng công thức nghiệm của phương trình bậc hai, có dạng:

\[
x = \frac{-b \pm \sqrt{b^2 - 4ac}}{2a}
\]

Trong phương trình của chúng ta, các hệ số là:
- \( a = 1 \)
- \( b = -5 \)
- \( c = 6 \)

Bước 1: Tính giá trị của \( b^2 - 4ac \).

\[
b^2 = (-5)^2 = 25
\]
\[
4ac = 4 \cdot 1 \cdot 6 = 24
\]
\[
b^2 - 4ac = 25 - 24 = 1
\]

Bước 2: Tính nghiệm của phương trình.

Áp dụng vào công thức nghiệm:

\[
x = \frac{-(-5) \pm \sqrt{1}}{2 \cdot 1}
\]
\[
x = \frac{5 \pm 1}{2}
\]

Bước 3: Tính các giá trị của \( x \).

- Nghiệm thứ nhất:
\[
x_1 = \frac{5 + 1}{2} = \frac{6}{2} = 3
\]

- Nghiệm thứ hai:
\[
x_2 = \frac{5 - 1}{2} = \frac{4}{2} = 2
\]

Vậy, phương trình \( x^2 - 5x + 6 = 0 \) có hai nghiệm:

\[
x_1 = 3 \quad \text{và} \quad x_2 = 2
\]


## 3.2. Few-shot CoT
Cách dùng: Cung cấp các ví dụ mẫu có kèm lập luận.


In [3]:
model = init_chat_model(
    model='gpt-4o-mini', 
    model_provider='openai',
    temperature=0.1
)


messages = [
    SystemMessage("""
Trong vai trò chuyên gia toán học, hãy giúp tôi giải phương trình bằng cách trình bày từng bước như các ví dụ sau:

**Ví dụ 1**:
Phương trình: x² - 5x + 6 = 0
Bước 1: Xác định hệ số a=1, b=-5, c=6
Bước 2: Tính delta Δ = b² - 4ac = (-5)² - 4*1*6 = 25 - 24 = 1
Bước 3: Vì Δ > 0, phương trình có 2 nghiệm phân biệt
Bước 4: x1 = (-b + √Δ)/(2a) = (5 + 1)/2 = 3
Bước 5: x2 = (-b - √Δ)/(2a) = (5 - 1)/2 = 2
→ Kết quả: x=3 hoặc x=2

**Ví dụ 2**:
Phương trình: 2x² + 4x - 6 = 0
Bước 1: Chia cả 2 vế cho 2 → x² + 2x - 3 = 0
Bước 2: Tính Δ = 2² - 4*1*(-3) = 4 + 12 = 16
Bước 3: √Δ = 4
Bước 4: x = [-2 ± 4]/2 → x1=1, x2=-3
→ Kết quả: x=1 hoặc x=-3
"""),
    HumanMessage('x^2 - 5x + 6 = 0'),
]

response = model.invoke(messages)
print(response.content) 

Phương trình: x² - 5x + 6 = 0

Bước 1: Xác định hệ số a=1, b=-5, c=6

Bước 2: Tính delta Δ = b² - 4ac = (-5)² - 4*1*6 = 25 - 24 = 1

Bước 3: Vì Δ > 0, phương trình có 2 nghiệm phân biệt

Bước 4: Tính nghiệm x1 = (-b + √Δ)/(2a) = (5 + 1)/2 = 3

Bước 5: Tính nghiệm x2 = (-b - √Δ)/(2a) = (5 - 1)/2 = 2

→ Kết quả: x=3 hoặc x=2


## 3.3. Auto-CoT 
* Tự động tạo các ví dụ suy luận bằng cách phân cụm câu hỏi và chọn mẫu đại diện.
* Quy trình:
    * Phân nhóm câu hỏi cùng loại
    * Tạo lập luận mẫu cho mỗi nhóm bằng Zero-shot CoT

# 4. Best Practices
* Sử dụng động từ chỉ dẫn: "Hãy phân tích từng bước", "Giải thích logic trước khi trả lời"
* Kết hợp với Few-shot: Cung cấp 2-3 ví dụ mẫu có lập luận chi tiết
* Kiểm soát độ dài: Dùng max_tokens phù hợp để đảm bảo đủ token cho các bước suy luận
* Tối ưu temperature:
    * Dùng temperature=0 cho tác vụ yêu cầu độ chính xác cao
    * temperature=0.7 cho tác vụ sáng tạo

# 5. Hạn chế
* Hiệu quả giảm với mô hình nhỏ (dưới 10B parameters)
* Tăng thời gian xử lý do phải sinh nhiều token hơn
* Cần thiết kế prompt cẩn thận để tránh lập luận lệch hướng